## Exercise results on i7-10875H

In [65]:
%%file tmp.precision.cpp

#include <iostream>
#include <cassert> // for assert
#include <cstdlib> // for rand
#include <valarray>

template< typename real >
void randomize( std::valarray<real> & collection )
 {
  srand(1) ;
  for ( auto & elem : collection )
   { elem = std::rand()/(RAND_MAX+1.)-0.5 ; }
 }

template< typename real >
real mean( std::valarray<real> const & collection )
 {
  real res {0.} ;
  for ( auto elem : collection )
   { res += elem ; }
  return res/collection.size() ;
 }

template< typename real >
void main_impl( unsigned long long size, unsigned long long repeat )
 {
  std::valarray<real> xs(0.,size) ;
  std::valarray<real> ys(0.,size) ;
  randomize(xs) ;
  while (repeat--)
    { ys = 0.1*xs + ys ; }
  real res = mean(ys) ;
  std::cout<<res<<std::endl ;
 }

int main( int argc, char * argv[] )
 {
  assert(argc==4) ;
  std::string precision(argv[1]) ;
  unsigned long long size = atoi(argv[2]) ;
  unsigned long long repeat = atoi(argv[3]) ;
  std::cout.precision(18) ;

  if (precision=="float") main_impl<float>(size,repeat) ;
  else if (precision=="double") main_impl<double>(size,repeat) ;
  else if (precision=="long") main_impl<long double>(size,repeat) ;
  else throw "unknown precision" ;
 }


Overwriting tmp.precision.cpp


In [106]:
%%file tmp.precision.sh
echo

opt=${1}
shift

rm -f tmp.precision.exe
g++ -O${opt} -march=native -std=c++17 tmp.precision.cpp -o tmp.precision.exe

./tmp.precision.exe $*

rm -f tmp.precision.py
echo "s = 0" >> tmp.precision.py
for i in 0 1 2 3 4 5 6 7 8 9 ; do
    \time -f "s += %U" -a -o ./tmp.precision.py ./tmp.precision.exe $*  >> /dev/null
done
echo "print(\"{:.4} s\".format(s/10.))" >> tmp.precision.py
python3 tmp.precision.py

echo

Overwriting tmp.precision.sh


In [107]:
! bash -l tmp.precision.sh 0 float 1024 100000


67.525482177734375
3.679 s



In [108]:
! bash -l tmp.precision.sh 0 double 1024 100000


67.5053500207703507
4.032 s



In [109]:
! bash -l tmp.precision.sh 0 long 1024 100000


67.5053500208378198
3.499 s



### With `g++ -O0 ...`

| Type   | time (s) | speedup |              result | significant digits |
| :------| -------: | ------: | ------------------: | -----------------: |
| float  |    3.307 |   0.034 | 67.525482177734375  |                  3 |
| double |    3.334 |   0.034 | 67.5053500207703507 |                 12 |
| long   |    3.285 |   0.034 | 67.5053500208378198 |                 18 |

In [110]:
! bash -l tmp.precision.sh 1 float 1024 100000


67.525482177734375
0.114 s



In [111]:
! bash -l tmp.precision.sh 1 double 1024 100000


67.5053500207703507
0.114 s



In [112]:
! bash -l tmp.precision.sh 1 long 1024 100000


67.5053500208378198
0.411 s



#### With `g++ -O1 ...`

| Type   | time (s) | speedup |              result | significant digits |
| :------| -------: | ------: | ------------------: | -----------------: |
| float  |      0.1 |    1.00 | 67.525482177734375  |                  3 |
| double |      0.1 |    1.00 | 67.5053500207703507 |                 12 |
| long   |      0.4 |    0.25 | 67.5053500208378198 |                 18 |

In [113]:
! bash -l tmp.precision.sh 2 float 1024 100000


67.525482177734375
0.114 s



In [114]:
! bash -l tmp.precision.sh 2 double 1024 100000


67.5053500207703507
0.115 s



In [115]:
! bash -l tmp.precision.sh 2 long 1024 100000


67.5053500208378198
0.753 s



#### With `g++ -O2 ...`

| Type   | time (s) | speedup |              result | significant digits |
| :------| -------: | ------: | ------------------: | -----------------: |
| float  |      0.1 |    1.00 | 67.525482177734375  |                  3 |
| double |      0.1 |    1.00 | 67.5053500207703507 |                 12 |
| long   |      0.7 |    0.14 | 67.5053500208378198 |                 18 |

In [118]:
! bash -l tmp.precision.sh 3 float 1024 100000


67.525482177734375
0.02 s



In [119]:
! bash -l tmp.precision.sh 3 double 1024 100000


67.5053500207703507
0.04 s



In [120]:
! bash -l tmp.precision.sh 3 long 1024 100000


67.5053500208378198
0.746 s



#### With `g++ -O3 ...`

| Type   | time (s) |  speedup |              result | significant digits |
| :------| -------: | -------: | ------------------: | -----------------: |
| float  |     0.02 |     5.00 | 67.525482177734375  |                  3 |
| double |     0.04 |     2.50 | 67.5053500207703507 |                 12 |
| long   |     0.70 |     0.14 | 67.5053500208378198 |                 18 |

In [123]:
! bash -l tmp.precision.sh fast float 1024 100000


67.5254364013671875
0.019 s



In [124]:
! bash -l tmp.precision.sh fast double 1024 100000


67.5053500207702939
0.042 s



In [125]:
! bash -l tmp.precision.sh fast long 1024 100000


67.5053500208378198
0.733 s



#### With `g++ -Ofast ...`

| Type   | time (s) |  speedup |              result | significant digits |
| :------| -------: | -------: | ------------------: | -----------------: |
| float  |     0.02 |     5.00 | 67.5254364013671875 |                  3 |
| double |     0.04 |     2.50 | 67.5053500207702939 |                 12 |
| long   |     0.70 |     0.14 | 67.5053500208378198 |                 18 |

© *CNRS 2021*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*